# Imports y configuraciones iniciales

In [1]:
import warnings

from typing import List

warnings.filterwarnings("ignore")

In [2]:
# Dejar el path principal como el anterior.
import sys
sys.path.append('../')

Seteamos las configuraciones del Logging

In [3]:
import logging

# Crear el logger
log = logging.getLogger(__name__)

# Setear el nivel del registro
log.setLevel(logging.DEBUG)

# Formato de los mensajes
formatter = logging.Formatter("%(levelname)s: (%(asctime)s) [%(filename)s: %(lineno)s] %(message)s")

if not log.hasHandlers():
    # Handlers
    file_handler = logging.FileHandler("logging.log")
    file_handler.setFormatter(formatter)  # Setear el formato del handler
    # Agregar el handler al logger
    # log.addHandler(file_handler)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    # Agregar el handler al logger
    log.addHandler(stream_handler)

Instalar la librería plot-likert y otras librerías útiles

In [4]:
# Librería para hacer gráficos Likert
# !pip install plot-likert

# Para obtener datos de excel
# !pip install openpyxl

# Para tener un transformador de data
# !pip install -U scikit-learn

# Para tener herramientas estadísticas
# !python -m pip install statsmodels

# Para tener Seaborn
# !pip install seaborn

Empezamos importando la librería para verificar que estuvo bien instalada.

In [5]:
import plot_likert

Importamos algunas librerías útiles para el resto del notebook

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [7]:
DATA_PATH = Path(r"..\data")

if not DATA_PATH.exists():
    log.info(f"Creando carpeta {DATA_PATH = }")
    DATA_PATH.mkdir()

In [8]:
QNA_COMPLETE_PATH = DATA_PATH / "questions-and-answers-complete.xlsx"
QNA_COMPLETE_PATH

WindowsPath('../data/questions-and-answers-complete.xlsx')

In [9]:
log.debug(f"Importando datos de {QNA_COMPLETE_PATH}")
df = pd.read_excel(QNA_COMPLETE_PATH)

df.head()

DEBUG: (2023-01-08 19:19:44,086) [2491434066.py: 1] Importando datos de ..\data\questions-and-answers-complete.xlsx


,Prom Cs,¿Los profesores de ciencias consideran los intereses de los alumnos para elaborar y organizar las evaluaciones?,De las actividades realizadas en clase ¿el profesor considera y respeta nuestros intereses?,¿El profesor retroalimenta y refuerza en todas las clases lo que hemos ido aprendiendo?,El profesor de la clase de ciencias ¿anota y explica el objetivo que se va a trabajar durante la clase?,"El profesor, al cierre de su clase ¿realiza preguntas para verificar si hemos aprendido?",Cuando el profesor da tareas ¿las explica claramente y nos da ejemplos que orientan para realizarlas?,Durante la actividad ¿el profesor circula explicando y resolviendo dudas?,"¿Se realizan salidas a terreno: museos, zoológicos y otros lugares que ayudan al aprendizaje?","Para hacer las clases, el profesor ¿nos lleva a otros espacios motivantes para que aprendamos mejor?",...,"Cuando me queda alguna duda de la clase, ¿me dirijo confiadamente al profesor a preguntar?","Cuando el profesor es cercano conmigo y mis compañeros, ¿nos da confianza y seguridad para preguntar todas las dudas?",Cuando tengo más cercanía y confianza con el profesor ¿aprendo mejor?,"¿Los profesores de ciencias ofrecen diversas actividades grupales de un mismo tema, para que los estudiantes, elijan en la que quieren trabajar?",¿Los profesores de ciencias integran a todos los estudiantes para que participen en las actividades grupales dentro de la sala de clases?,"¿Los profesores de ciencias integran a todos los estudiantes en grupos, para que participen en las actividades de investigación, fuera del colegio?","¿Los profesores, muestran interés por reforzar a los alumnos que más les cuesta?","Cuando el profesor nos hace realizar actividades variadas, en ciencias ¿comprendo mejor la materia?",¿Los profesores de ciencias integran a todos los estudiantes para que participen en las actividades prácticas de laboratorio?,"En las clases de ciencias, ¿algunos alumnos son evaluados con carpetas de actividades realizadas en el colegio y fuera de él?"
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempr

In [10]:
questions = list(df.columns)[1:]
questions

['¿Los profesores de ciencias consideran los intereses de los alumnos para elaborar y organizar las evaluaciones?',
 'De las actividades realizadas en clase ¿el profesor considera y respeta nuestros intereses?',
 '¿El profesor retroalimenta y refuerza en todas las clases lo que hemos ido aprendiendo?',
 'El profesor de la clase de ciencias ¿anota y explica el objetivo que se va a trabajar durante la clase?',
 'El profesor, al cierre de su clase ¿realiza preguntas para verificar si hemos aprendido?',
 'Cuando el profesor da tareas ¿las explica claramente y nos da ejemplos que orientan para realizarlas?',
 'Durante la actividad ¿el profesor circula explicando y resolviendo dudas?',
 '¿Se realizan salidas a terreno: museos, zoológicos y otros lugares que ayudan al aprendizaje?',
 'Para hacer las clases, el profesor ¿nos lleva a otros espacios motivantes para que aprendamos mejor?',
 'El profesor de ciencias ¿presenta en sus clases material atractivo para aprender?',
 '¿Los profesores comp

In [11]:
import copy


log.debug("Renombrando columnas")
questions_copy = copy.copy(questions)
for i in range(len(questions)):
    # questions_copy[i] = f"P{i+1}.: " + questions[i]
    questions_copy[i] = f"P{i+1}"

df.columns = questions = ["Notas"] +  questions_copy


df.head()

DEBUG: (2023-01-08 19:19:44,592) [2321751420.py: 4] Renombrando columnas


,Notas,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca


# Reescalamiento y codificación

In [12]:
scales = [
    "Nunca",
    "Rara vez",
    "Ocasionalmente",
    "Casi siempre",
    "Siempre, en todas las clases"
]

Codificamos las respuestas para hacer la regresión

In [13]:
from collections import OrderedDict
df_enc = OrderedDict()

encoder = {name: i-2 for i, name in enumerate(scales)}
for col_name in questions_copy:
    df_enc[col_name] = df[col_name].map(encoder)

df_enc["Notas binaria"] = (df["Notas"] >= 6).astype(int)
df_enc["Notas continuas"] = (df["Notas"] - min(df["Notas"])) / (max(df["Notas"]) - min(df["Notas"]))

df_enc = pd.DataFrame(df_enc)
df_enc

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P24,P25,P26,P27,P28,P29,P30,P31,Notas binaria,Notas continuas
0,0,1,2,2,2,1,2,-2,-1,1,...,2,1,1,-1,1,2,2,-2,1,1.000000
1,2,2,2,2,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,1,0.958333
2,2,2,2,2,0,2,2,-2,0,2,...,2,2,2,-2,2,2,2,0,1,0.708333
3,1,1,-1,1,1,1,0,-2,0,0,...,2,0,1,2,-1,0,-1,-2,0,0.083333
4,2,2,2,2,2,2,1,-2,0,2,...,2,2,2,0,2,2,2,-2,1,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1,2,1,2,1,1,1,-1,1,2,...,1,1,1,1,2,1,1,1,0,0.166667
82,0,-1,1,2,0,2,-1,-1,-1,-2,...,-1,0,0,0,0,-2,-2,-2,0,0.541667
83,2,0,1,2,2,0,2,-2,-1,2,...,-1,0,2,1,2,1,2,-2,1,0.916667
84,2,2,2,2,2,2,2,-1,-1,2,...,2,1,2,2,2,2,2,0,1,0.833333


In [14]:
df["Notas"] >= 6

0      True
1      True
2      True
3     False
4      True
      ...  
81    False
82    False
83     True
84     True
85    False
Name: Notas, Length: 86, dtype: bool

In [15]:
df_enc.describe(percentiles=[i / 10 for i in range(10)])

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P24,P25,P26,P27,P28,P29,P30,P31,Notas binaria,Notas continuas
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,...,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,0.686047,1.023256,1.360465,1.662791,1.034884,1.174419,1.302326,-1.639535,-1.093023,0.837209,...,1.139535,0.500000,1.279070,-0.046512,1.162791,0.953488,0.279070,-0.988372,0.453488,0.561047
std,1.031978,0.920061,0.866539,0.661735,0.975556,1.019710,0.946446,0.880011,1.047438,1.125746,...,1.097192,1.317306,0.862979,1.637121,0.931292,1.072731,1.606246,1.231873,0.500752,0.281416
min,-2.000000,-1.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,0.000000,0.000000
0%,-2.000000,-1.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,0.000000,0.000000
10%,-1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000,-1.000000,...,-0.500000,-1.000000,0.000000,-2.000000,0.000000,0.000000,-2.000000,-2.000000,0.000000,0.166667
20%,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000,0.000000,...,0.000000,-1.000000,1.000000,-2.000000,0.000000,0.000000,-2.000000,-2.000000,0.000000,0.250000
30%,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,-2.000000,-2.000000,0.000000,...,1.000000,0.000000,1.000000,-1.500000,1.000000,0.000000,-1.000000,-2.000000,0.000000,0.375000
40%,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,-2.000000,-2.000000,1.000000,...,1.000000,0.000000,1.000000,-1.000000,1.000000,1.000000,0.000000,-2.000000,0.000000,0.500000
50%,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,-2.000000,-1.000000,1.000000,...,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,-1.000000,0.000000,0.604167


# Separación entrenamiento-test

In [16]:
import statsmodels.api as sm

Asignamos como variable endógena (la variable dependiente) las notas (recordando que es 1 si la nota es mayor a 6 y 0 en otro caso).

In [17]:
df_enc.endog = np.array(df_enc.pop("Notas binaria")).reshape((-1, 1))

In [18]:
df_enc.endog_lin = np.array(df_enc.pop("Notas continuas")).reshape((-1, 1))

# df_enc.endog

In [19]:
np.mean(df_enc.endog)

0.45348837209302323

In [20]:
np.mean(df_enc.endog_lin)

0.5610465116279069

Y asignamos como variables exógenas las respuestas a las preguntas.

In [21]:
df_enc.exog = sm.add_constant(df_enc)
df_enc.exog

,const,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,1.0,0,1,2,2,2,1,2,-2,-1,...,1,2,2,1,1,-1,1,2,2,-2
1,1.0,2,2,2,2,1,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,1.0,2,2,2,2,0,2,2,-2,0,...,2,2,2,2,2,-2,2,2,2,0
3,1.0,1,1,-1,1,1,1,0,-2,0,...,-2,1,2,0,1,2,-1,0,-1,-2
4,1.0,2,2,2,2,2,2,1,-2,0,...,2,2,2,2,2,0,2,2,2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1.0,1,2,1,2,1,1,1,-1,1,...,1,1,1,1,1,1,2,1,1,1
82,1.0,0,-1,1,2,0,2,-1,-1,-1,...,-2,-1,-1,0,0,0,0,-2,-2,-2
83,1.0,2,0,1,2,2,0,2,-2,-1,...,2,1,-1,0,2,1,2,1,2,-2
84,1.0,2,2,2,2,2,2,2,-1,-1,...,1,1,2,1,2,2,2,2,2,0


In [22]:
from sklearn.model_selection import train_test_split

df_enc.exog_train, df_enc.exog_test, df_enc.endog_train, df_enc.endog_test = train_test_split(
    df_enc.exog, df_enc.endog, test_size=0.10, random_state=42
)

df_enc.exog_lin_train, df_enc.exog_lin_test, df_enc.endog_lin_train, df_enc.endog_lin_test = train_test_split(
    df_enc.exog, df_enc.endog_lin, test_size=0.10, random_state=42
)
print(f"{len(df_enc.exog_train) = },\n{len(df_enc.exog_test) = },\n{len(df_enc.endog_train) = },\n{len(df_enc.endog_test) = }")

len(df_enc.exog_train) = 77,
len(df_enc.exog_test) = 9,
len(df_enc.endog_train) = 77,
len(df_enc.endog_test) = 9


En el índice estarían los alumnos para el test

In [23]:
df_enc.exog_test

,const,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
75,1.0,1,0,1,2,-1,0,0,-2,-1,...,1,1,2,-1,2,-1,1,0,-1,-1
0,1.0,0,1,2,2,2,1,2,-2,-1,...,1,2,2,1,1,-1,1,2,2,-2
70,1.0,2,2,2,2,2,2,2,-2,-1,...,2,2,2,2,2,-2,2,2,-2,-2
22,1.0,2,2,2,2,2,2,2,-1,0,...,2,2,2,2,2,2,2,2,2,0
12,1.0,-1,-1,0,0,1,-2,2,-2,-2,...,0,1,1,2,1,-2,1,1,2,-2
56,1.0,-1,1,2,1,-1,1,1,-2,-2,...,-1,1,1,-1,2,-2,1,1,-2,-2
10,1.0,0,1,2,2,1,0,1,-2,-1,...,2,2,2,2,1,2,1,1,0,0
18,1.0,2,2,2,2,2,2,2,-2,-1,...,2,2,2,2,2,-2,2,2,2,-2
4,1.0,2,2,2,2,2,2,1,-2,0,...,2,2,2,2,2,0,2,2,2,-2


In [24]:
df["Notas"][df_enc.exog_test.index]

75    5.1
0     6.8
70    5.8
22    6.4
12    6.2
56    5.1
10    6.6
18    5.7
4     6.5
Name: Notas, dtype: float64

# Primer modelo de regresión logística

Creamos el modelo y lo ajustamos. Luego revisamos los parámetros aprendidos

In [25]:
logit_mod = sm.Logit(df_enc.endog_train, df_enc.exog_train)

logit_res = logit_mod.fit()

logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.454224
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   77
Model:                          Logit   Df Residuals:                       45
Method:                           MLE   Df Model:                           31
Date:                Sun, 08 Jan 2023   Pseudo R-squ.:                  0.3382
Time:                        19:19:47   Log-Likelihood:                -34.975
converged:                       True   LL-Null:                       -52.845
Covariance Type:            nonrobust   LLR p-value:                    0.2554
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3032      1.548     -0.842      0.400      -4.337       1.731
P1             0.7725      0.648      1.192      0.233      -0.497       2.042
P2            -0.3807      0.509     -0.748      0.455      -1.379       0.617
P3            -0.1377      0.718     -0.192      0.848      -1.544       1.269
P4            -0.7811      0.744     -1.050      0.294      -2.239       0.677
P5             0.6497      0.578      1.125      0.261      -0.482       1.782
P6            -0.0177      0.778     -0.023      0.982      -1.543       1.508
P7            -0.3664      0.705     -0.520      0.603      -1.748       1.016
P8             0.1324      0.579      0.229      0.819      -1.003       1.268
P9             0.5060      0.550      0.920      0.358      -0.573       1.585
P10           -0.7764      0.591     -1.313      0.189      -1.935       0.383
P11            1.2946      1.060      1.221      0.222      -0.783       3.372
P12           -1.2296      0.693     -1.774      0.076      -2.588       0.129
P13           -0.5238      1.056     -0.496      0.620      -2.594       1.547
P14           -0.2047      0.313     -0.654      0.513      -0.818       0.409
P15           -0.7587      0.447     -1.698      0.090      -1.635       0.117
P16           -0.3590      0.399     -0.901      0.368      -1.140       0.422
P17           -0.5894      0.499     -1.181      0.237      -1.567       0.388
P18            0.4668      0.667      0.700      0.484      -0.841       1.775
P19           -0.4329      0.669     -0.647      0.517      -1.743       0.878
P20           -0.3038      0.348     -0.874      0.382      -0.985       0.377
P21            1.8286      0.856      2.135      0.033       0.150       3.507
P22            0.0274      0.540      0.051      0.960      -1.032       1.087
P23            1.7733      0.736      2.408      0.016       0.330       3.216
P24           -0.7691      0.588     -1.308      0.191      -1.921       0.383
P25           -0.4761      0.421     -1.131      0.258      -1.301       0.349
P26           -0.1501      0.586     -0.256      0.798      -1.299       0.998
P27           -0.3750      0.351     -1.070      0.285      -1.062       0.312
P28           -0.1911      0.708     -0.270      0.787      -1.579       1.197
P29            0.9949      0.669      1.487      0.137      -0.317       2.307
P30            0.9098      0.329      2.767      0.006       0.265       1.554
P31            0.6204      0.533      1.164      0.245      -0.425       1.665
==============================================================================
"""

A partir de aquí notamos que las preguntas 21, 23 y 30 son significativas al $5\%$. Revisemos cuál es el error cuadrático medio para obtener una comparativa.

In [26]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_test.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog_test),
)

0.2777511807362247

In [27]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_train.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog_train),
)

0.1465087304735915

In [28]:
mean_squared_error(
    y_true=df_enc.endog_test.reshape(-1,),
    y_pred=(logit_res.predict(df_enc.exog_test) >= 0.5).astype(int),
)

0.4444444444444444

In [29]:
mean_squared_error(
    y_true=df_enc.endog_train.reshape(-1,),
    y_pred=(logit_res.predict(df_enc.exog_train) >= 0.5).astype(int),
)

0.18181818181818182

Notando que 
\begin{align}
0\leq (y_{true} - \hat y_{pred}) ^2  \leq 1
\end{align}
Pues el valor de la predicción siempre está entre en el intervalo $[0, 1]$, notamos que el este error resulta ser satisfactorio, pues de no ser así, al menos tendería a $0.5$, cosa que no sucede en este caso.

In [29]:

alumnos = [20, 81, 10, 36, 39]
alumnos

[20, 81, 10, 36, 39]

In [30]:
logit_res.predict(df_enc.exog.iloc[alumnos])

20    0.112787
81    0.184250
10    0.476992
36    0.420288
39    0.396426
dtype: float64

In [31]:
(logit_res.predict(df_enc.exog.iloc[alumnos]) >= 0.5).astype(int)

20    0
81    0
10    0
36    0
39    0
dtype: int32

In [32]:
df_enc.endog.reshape(-1,)[alumnos]

array([1, 0, 1, 0, 0])

In [33]:
df.Notas[alumnos]

20    6.3
81    4.8
10    6.6
36    4.9
39    5.0
Name: Notas, dtype: float64

In [ ]:
# Regresión con variables

Se determinó a través del coeficiente VIF las variables que permitirán tener una buena correlación para hacer una regresión. Se utilizarán estas variables en el modelo. Empezaremos haciendo una regresión con aquellos que tengan VIF menor o igual a 5 (como se sugiere).

## $VIF \leq 5$

In [34]:
vif_leq_5 = [
    "P14",
    "P15",
    "P16",
    "P17",
    "P20",
    "P25",
    "P27",
    "P30",
    "P31",
]

In [35]:
logit_mod_VIF_5 = sm.Logit(df_enc.endog_train, df_enc.exog_train[vif_leq_5])

logit_res = logit_mod_VIF_5.fit()

logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.614307
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   77
Model:                          Logit   Df Residuals:                       68
Method:                           MLE   Df Model:                            8
Date:                Sun, 08 Jan 2023   Pseudo R-squ.:                  0.1049
Time:                        16:16:46   Log-Likelihood:                -47.302
converged:                       True   LL-Null:                       -52.845
Covariance Type:            nonrobust   LLR p-value:                    0.1968
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
P14           -0.0326      0.200     -0.163      0.871      -0.425       0.360
P15           -0.2268      0.260     -0.873      0.383      -0.736       0.282
P16            0.2008      0.228      0.881      0.378      -0.246       0.648
P17           -0.1464      0.249     -0.588      0.557      -0.635       0.342
P20            0.1029      0.184      0.561      0.575      -0.257       0.463
P25           -0.3190      0.244     -1.307      0.191      -0.798       0.159
P27           -0.0888      0.212     -0.418      0.676      -0.505       0.327
P30            0.4281      0.194      2.203      0.028       0.047       0.809
P31            0.3718      0.279      1.333      0.182      -0.175       0.918
==============================================================================
"""

In [36]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_test.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog_test[vif_leq_5]),
)

0.19741805328698062

In [37]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_train.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog_train[vif_leq_5]),
)

0.2134917375428169

In [38]:
mean_squared_error(
    y_true=df_enc.endog_test.reshape(-1,),
    y_pred=(logit_res.predict(df_enc.exog_test[vif_leq_5]) >= 0.5).astype(int),
)

0.4444444444444444

## $VIF \leq 12$

In [39]:
vif_leq_12 = [
    "P1",
    "P2",
    "P5",
    "P6",
    "P7",
    "P9",
    "P10",
    "P12",
    "P14",
    "P15",
    "P16",
    "P17",
    "P18",
    "P19",
    "P20",
    "P21",
    "P22",
    "P23",
    "P24",
    "P25",
    "P26",
    "P27",
    "P28",
    "P29",
    "P30",
]

In [40]:
logit_mod_VIF_12 = sm.Logit(df_enc.endog_train, df_enc.exog_train[vif_leq_12])

logit_res = logit_mod_VIF_12.fit()

logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.542067
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   77
Model:                          Logit   Df Residuals:                       52
Method:                           MLE   Df Model:                           24
Date:                Sun, 08 Jan 2023   Pseudo R-squ.:                  0.2102
Time:                        16:16:46   Log-Likelihood:                -41.739
converged:                       True   LL-Null:                       -52.845
Covariance Type:            nonrobust   LLR p-value:                    0.5666
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
P1             0.4739      0.475      0.997      0.319      -0.457       1.405
P2            -0.4200      0.432     -0.973      0.330      -1.266       0.426
P5             0.3948      0.393      1.005      0.315      -0.375       1.165
P6            -0.1691      0.589     -0.287      0.774      -1.323       0.985
P7            -0.2323      0.554     -0.419      0.675      -1.318       0.854
P9             0.6836      0.368      1.859      0.063      -0.037       1.404
P10           -0.4889      0.426     -1.146      0.252      -1.325       0.347
P12           -0.3311      0.497     -0.667      0.505      -1.305       0.642
P14            0.0996      0.245      0.407      0.684      -0.380       0.580
P15           -0.1567      0.307     -0.510      0.610      -0.759       0.445
P16           -0.0412      0.314     -0.131      0.896      -0.656       0.573
P17           -0.2466      0.390     -0.633      0.527      -1.010       0.517
P18            0.1261      0.513      0.246      0.806      -0.880       1.133
P19           -0.2089      0.493     -0.424      0.671      -1.175       0.757
P20           -0.1822      0.267     -0.683      0.495      -0.705       0.341
P21            0.9422      0.584      1.614      0.107      -0.202       2.087
P22            0.3387      0.398      0.850      0.395      -0.442       1.120
P23            0.7429      0.507      1.465      0.143      -0.251       1.737
P24           -0.3256      0.471     -0.691      0.490      -1.249       0.598
P25           -0.4223      0.322     -1.310      0.190      -1.054       0.210
P26           -0.4150      0.488     -0.850      0.395      -1.372       0.542
P27           -0.2601      0.270     -0.964      0.335      -0.789       0.269
P28           -0.3356      0.579     -0.579      0.562      -1.471       0.800
P29            0.5106      0.473      1.081      0.280      -0.416       1.437
P30            0.6100      0.249      2.454      0.014       0.123       1.097
==============================================================================
"""

In [41]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_test.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog_test[vif_leq_12]),
)

0.22498938681673067

In [42]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_train.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog_train[vif_leq_12]),
)

0.17898916378557625

In [43]:
mean_squared_error(
    y_true=df_enc.endog_test.reshape(-1,),
    y_pred=(logit_res.predict(df_enc.exog_test[vif_leq_12]) >= 0.5).astype(int),
)

0.3333333333333333

## Regresión lineal

Intentaremos con probar una regresión lineal, en caso de que tenga un mejor ajuste o un mejor error cuadrático medio

## $VIF \leq 5$

In [44]:
vif_leq_5 = [
    "P14",
    "P15",
    "P16",
    "P17",
    "P20",
    "P25",
    "P27",
    "P30",
    "P31",
]

In [45]:
ols_mod_VIF_5 = sm.OLS(df_enc.endog_lin_train, df_enc.exog_lin_train[vif_leq_5])

ols_res = ols_mod_VIF_5.fit()

ols_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.649
Model:                            OLS   Adj. R-squared (uncentered):              0.602
Method:                 Least Squares   F-statistic:                              13.96
Date:                Sun, 08 Jan 2023   Prob (F-statistic):                    2.03e-12
Time:                        16:16:46   Log-Likelihood:                         -31.556
No. Observations:                  77   AIC:                                      81.11
Df Residuals:                      68   BIC:                                      102.2
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
P14           -0.0336      0.035     -0.958      0.342      -0.104       0.036
P15           -0.1048      0.045     -2.346      0.022      -0.194      -0.016
P16            0.0996      0.040      2.503      0.015       0.020       0.179
P17            0.1210      0.044      2.779      0.007       0.034       0.208
P20           -0.0510      0.032     -1.577      0.120      -0.116       0.014
P25           -0.0142      0.043     -0.333      0.740      -0.099       0.071
P27            0.0377      0.037      1.025      0.309      -0.036       0.111
P30            0.0818      0.033      2.453      0.017       0.015       0.148
P31           -0.0405      0.046     -0.879      0.382      -0.132       0.051
==============================================================================
Omnibus:                        2.776   Durbin-Watson:                   1.328
Prob(Omnibus):                  0.250   Jarque-Bera (JB):                1.814
Skew:                           0.146   Prob(JB):                        0.404
Kurtosis:                       2.307   Cond. No.                         3.89
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_lin_test.reshape(-1,),
    y_pred=ols_res.predict(df_enc.exog_lin_test[vif_leq_5]),
)

0.10684541888103903

In [47]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_lin_train.reshape(-1,),
    y_pred=ols_res.predict(df_enc.exog_lin_train[vif_leq_5]),
)

0.13288894228598822

**ADVERTENCIA**: Esta métrica no puede ser comparable con el anterior, pues el tipo de modelo que se comparan son totalmente distintos.

Comparemos los datos predichos con los reales

In [48]:
# Datos predichos
y_pred = np.array(ols_res.predict(df_enc.exog_lin_test[vif_leq_5]))
y_pred

array([ 0.30764692,  0.48675426,  0.27055484,  0.77371918,  0.71865365,
       -0.09663868,  0.49157452,  0.96062228,  0.57741438])

In [49]:
# Reales
y_train = df_enc.endog_lin_test.reshape(-1,)
y_train

array([0.29166667, 1.        , 0.58333333, 0.83333333, 0.75      ,
       0.29166667, 0.91666667, 0.54166667, 0.875     ])

In [50]:
y_train - y_pred

array([-0.01598026,  0.51324574,  0.3127785 ,  0.05961415,  0.03134635,
        0.38830534,  0.42509215, -0.41895561,  0.29758562])

Se nota que hay varios que están se alejan del real por un 0.5, lo cuál es malo al pensar que tenemos valores entre el 0 (nota mínima) y el 1 (nota máxima).

## $VIF \leq 12$

In [51]:
vif_leq_12 = [
    "P1",
    "P2",
    "P5",
    "P6",
    "P7",
    "P9",
    "P10",
    "P12",
    "P14",
    "P15",
    "P16",
    "P17",
    "P18",
    "P19",
    "P20",
    "P21",
    "P22",
    "P23",
    "P24",
    "P25",
    "P26",
    "P27",
    "P28",
    "P29",
    "P30",
]

In [52]:
ols_mod_VIF_12 = sm.OLS(df_enc.endog_lin_train, df_enc.exog_lin_train[vif_leq_12])

ols_res = ols_mod_VIF_12.fit()

ols_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.828
Model:                            OLS   Adj. R-squared (uncentered):              0.745
Method:                 Least Squares   F-statistic:                              10.01
Date:                Sun, 08 Jan 2023   Prob (F-statistic):                    2.58e-12
Time:                        16:16:46   Log-Likelihood:                         -4.0945
No. Observations:                  77   AIC:                                      58.19
Df Residuals:                      52   BIC:                                      116.8
Df Model:                          25                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
P1             0.0528      0.063      0.834      0.408      -0.074       0.180
P2            -0.0023      0.053     -0.043      0.966      -0.108       0.103
P5             0.0584      0.051      1.151      0.255      -0.043       0.160
P6            -0.0036      0.066     -0.054      0.957      -0.135       0.128
P7            -0.0386      0.069     -0.559      0.578      -0.177       0.100
P9             0.0031      0.047      0.067      0.947      -0.091       0.097
P10           -0.0395      0.055     -0.722      0.473      -0.149       0.070
P12           -0.1149      0.061     -1.875      0.066      -0.238       0.008
P14           -0.0084      0.031     -0.269      0.789      -0.071       0.055
P15           -0.0620      0.040     -1.548      0.128      -0.142       0.018
P16           -0.0179      0.041     -0.432      0.668      -0.101       0.065
P17           -0.0375      0.052     -0.723      0.473      -0.142       0.067
P18            0.0169      0.066      0.255      0.800      -0.116       0.149
P19            0.0101      0.066      0.154      0.879      -0.122       0.142
P20           -0.0190      0.034     -0.567      0.573      -0.086       0.048
P21            0.2149      0.073      2.960      0.005       0.069       0.361
P22            0.0238      0.052      0.460      0.648      -0.080       0.127
P23            0.1639      0.064      2.546      0.014       0.035       0.293
P24           -0.0550      0.062     -0.893      0.376      -0.178       0.069
P25            0.0063      0.040      0.157      0.876      -0.074       0.087
P26            0.0390      0.060      0.649      0.519      -0.082       0.160
P27           -0.0509      0.035     -1.437      0.157      -0.122       0.020
P28            0.0047      0.074      0.064      0.950      -0.144       0.153
P29            0.0145      0.060      0.240      0.811      -0.107       0.136
P30            0.0813      0.030      2.708      0.009       0.021       0.141
==============================================================================
Omnibus:                        2.314   Durbin-Watson:                   1.560
Prob(Omnibus):                  0.314   Jarque-Bera (JB):                1.858
Skew:                           0.008   Prob(JB):                        0.395
Kurtosis:                       3.761   Cond. No.                         16.4
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_lin_test.reshape(-1,),
    y_pred=ols_res.predict(df_enc.exog_lin_test[vif_leq_12]),
)

0.14578146192990576

In [54]:
from sklearn.metrics import mean_squared_error


mean_squared_error(
    y_true=df_enc.endog_lin_train.reshape(-1,),
    y_pred=ols_res.predict(df_enc.exog_lin_train[vif_leq_12]),
)

0.06511981481933952